In [1]:
 
print("Ok")

Ok


In [2]:
%pwd

'/Users/myhome/prince/End_-to-End_-medical_bot/research'

In [3]:
import os
os.chdir("../")

In [48]:
%pwd

'/Users/myhome/prince/End_-to-End_-medical_bot'

In [49]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [50]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [51]:
extracted_data=load_pdf_file(data='Data/')

In [52]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [53]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2940


In [54]:
from langchain.embeddings import HuggingFaceEmbeddings

In [55]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [16]:
pip install -U sentence-transformers

  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2
Note: you may need to restart the kernel to use updated packages.


In [56]:
embeddings = download_hugging_face_embeddings()

In [57]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [58]:
from dotenv import load_dotenv
load_dotenv()

True

In [61]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [62]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [65]:
from langchain.vectorstores import Pinecone



In [66]:


docsearch = Pinecone.from_documents(
    documents=text_chunks,  # your document chunks
    index_name=index_name,
    embedding=embeddings
)

In [67]:
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [68]:
docsearch

In [39]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [40]:
retrieved_docs = retriever.invoke("What is eczema?")



In [41]:
retrieved_docs

[Document(metadata={'page': 126.0, 'source': 'Data/Medical_book.pdf'}, page_content='Disease Focus 101\nDiseases and Conditions—cont’d\nTerm Definition\necchymosis\n˘ek-˘ ı-MŌ-s˘ ıs\neczema\n˘EK-z ˘e-mă\nerythema\n˘er-˘ ı-THĒ-mă\neschar\n˘ES-kăr\nimpetigo\n˘ ım-p˘e-TĪ-gō\nkeratosis\nk˘er-ă-TŌ-s˘ ıs\nkerat: horny tissue, hard; cornea\n-osis: abnormal condition; increase \n(used primarily with blood cells)\nlentigo\nl˘en-TĪ-gō\npallor\nPĂL-or\npediculosis\np˘e-d˘ ık-ū-LŌ-s˘ ıs\npedicul: lice\n-osis: abnormal condition; increase \n(used primarily with blood cells)'),
 Document(metadata={'page': 126.0, 'source': 'Data/Medical_book.pdf'}, page_content='Skin discoloration consisting of a large, irregularly formed hemorrhagic\narea with colors changing from bluish black to greenish brown or yellow;\ncommonly called a bruise (See Fig. 5-13.)\nChronic inflammatory skin condition that is characterized by erythema,\npapules, vesicles, pustules, scales, crusts, and scabs and accompanied by\nintens

In [42]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [44]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [45]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])



Acromegaly is a disorder characterized by excessive growth hormone (GH) secretion in adults, resulting in enlargement of bones in the extremities and head. Gigantism is a condition that occurs when there is hypersecretion of GH during childhood and puberty, causing excessive growth and resulting in a person being significantly taller than average.


In [46]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])



I don't know.
